# 1. GCI 데이터 변환

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import os
from datetime import datetime, timedelta
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import csv

In [2]:
#기본 경로 선언
source_path = '../data1/'
output_path = '../data2/'

gci_filenm = 'raw_gci.csv'
pds_filenm = 'raw_index_pds.csv'
var_filenm = '로이터_변수_데이터set_180605.xlsx'
idx_filenm = '로이터_지표_인도_오스트리아_인디아_아프리카_180614_원본.xlsx'

gci_path = os.path.join(source_path, gci_filenm)
pds_path = os.path.join(source_path, pds_filenm)
var_path = os.path.join(source_path, var_filenm)
idx_path = os.path.join(source_path, idx_filenm)

gci_output_path = os.path.join(output_path, 'weekly_gci.csv')
pds_output_path = os.path.join(output_path, 'weekly_pds.csv')
var_output_path = os.path.join(output_path, 'var_reuter_set.csv')
idx_output_path = os.path.join(output_path, 'idx_reuter_coal.csv')

In [3]:
# csv 파일 불러오기
df_gci = pd.read_csv(gci_path, parse_dates=True)
# 컬럼명 변경, 데이터(Date)형식 변경 및 Index 컬럼 변경
df_gci.columns = ['Date', 'NEWC']
df_gci['Date'] = pd.to_datetime(df_gci['Date'])
df_gci = df_gci.set_index('Date')

#현재 변수에 있는 내용 보여주기
df_gci.head()

,NEWC
Date,
2003-07-04,24.63
2003-07-11,24.59
2003-07-18,24.82
2003-07-25,24.89
2003-08-01,24.91


In [4]:
#변경된 csv 저장
df_gci.to_csv(gci_output_path)

# 2. KoreaPDS 데이터 변환

In [5]:
# csv 파일 불러오기
df_pds = pd.read_csv(pds_path, index_col=0, parse_dates=True)

#pds csv의 row와 column수 출력
df_pds.shape

(602, 107)

In [6]:
#Bloomberg, KoreaPDF 지수 제거
df_pds = df_pds.drop(columns=df_pds.filter(regex='PDS|Bloomberg'))
df_pds.shape

(602, 98)

In [7]:
df_pds.head()

,WorldBank-Agriculture Index,IMF-Crude Oil(petroleum) Price index,CRB BLS Spot Index,철근 [SHFE] 2018.10,LMEX Index,GSCI Light Energy Index (CPW 4),GSCI Livestock,WorldBank-Energy Index,중후판 12(mm)x8(ft)x20(ft) [국내(1차 유통가)] 현물,원당 raw sugar [ICE(NYBOT)] 2018.10,...,Baltic Dirty Tanker Index (BDTI),GSCI Precious Metals,백금 [LPPM] 현물,팔라듐 [LPPM] 현물,IMF-Metals Price Index,IMF-Fuel (energy) Index,Howe Robinson Container Index,GSCI Petroleum,CRB BLS Metals Sub-Index,CRB BLS Fat & Oils Sub-Index
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-05,73.6850,102.4100,359.17,3577.0,3459.10,550.84,267.27,74.755,700000.0,11.07,...,1217.8,1988.83,1132.8,330.1,171.5875,104.0725,1027.0,329.48,686.3950,273.0175
2007-01-12,74.1500,104.3000,363.91,3577.0,3439.80,550.84,267.27,76.300,700000.0,10.81,...,1208.2,1988.83,1147.2,334.4,172.3950,105.7850,1035.0,329.48,695.0500,275.6350
2007-01-19,74.6150,106.1900,367.04,3577.0,3579.16,550.84,267.27,77.845,700000.0,10.78,...,1241.6,1988.83,1168.6,347.1,173.2025,107.4975,1060.4,329.48,703.7050,278.2525
2007-01-26,75.0800,108.0800,371.99,3577.0,3465.72,550.84,267.27,79.390,700000.0,10.57,...,1203.8,1988.83,1171.8,337.2,174.0100,109.2100,1086.7,329.48,712.3600,280.8700
2007-02-02,75.0525,109.5225,365.30,3577.0,3378.16,550.84,267.27,79.930,700000.0,10.31,...,1174.0,1988.83,1185.8,339.2,176.4475,110.4300,1093.4,329.48,727.5525,291.6200


In [8]:
#인코딩 후 저장
df_pds.to_csv(pds_output_path, encoding='ms949')

# 3. Reuter 데이터 변환

In [9]:
df_pre_reuter = pd.read_excel(var_path)
df_reuter1 = df_pre_reuter.iloc[1:, 24:26]
df_reuter1.columns = ['Date', 'OILFUTURES=']
df_reuter2 = df_pre_reuter.iloc[1:, 0:2]
df_reuter2.columns = ['Date', 'USFFTARGET= CHT']
df_reuter3 = df_pre_reuter.iloc[1:, 3:5]
df_reuter3.columns = ['Date', 'USD=']
df_reuter4 = df_pre_reuter.iloc[1:, 6:8]
df_reuter4.columns = ['Date', 'XAU=']
df_reuter5 = df_pre_reuter.iloc[1:, 9:11]
df_reuter5.columns = ['Date', 'CNPMIC=ECI']
df_reuter6 = df_pre_reuter.iloc[1:, 12:14]
df_reuter6.columns = ['Date', 'JPRPMI=ECI']
df_reuter7 = df_pre_reuter.iloc[1:, 15:17]
df_reuter7.columns = ['Date', 'KRBUSI=ECI']
df_reuter8 = df_pre_reuter.iloc[1:, 18:20]
df_reuter8.columns = ['Date', 'USPMI=ECI']
df_reuter9 = df_pre_reuter.iloc[1:, 21:23]
df_reuter9.columns = ['Date', 'WTI=']
df_reuter10 = df_pre_reuter.iloc[1:, 27:29]
df_reuter10.columns = ['Date', 'GCI=']
df_reuter11 = df_pre_reuter.iloc[1:, 30:32]
df_reuter11.columns = ['Date', 'CCE=STOCK']
df_reuter = pd.merge(df_reuter1, df_reuter2, how='left')
df_reuter = pd.merge(df_reuter, df_reuter3, how='left')
df_reuter = pd.merge(df_reuter, df_reuter4, how='left')
df_reuter = pd.merge(df_reuter, df_reuter5, how='left')
df_reuter = pd.merge(df_reuter, df_reuter6, how='left')
df_reuter = pd.merge(df_reuter, df_reuter7, how='left')
df_reuter = pd.merge(df_reuter, df_reuter8, how='left')
df_reuter = pd.merge(df_reuter, df_reuter9, how='left')
df_reuter = pd.merge(df_reuter, df_reuter10, how='left')
df_reuter = pd.merge(df_reuter, df_reuter11, how='left')
df_reuter = df_reuter.drop_duplicates(['Date'], keep='last')
df_reuter = df_reuter.set_index('Date')
df_reuter = df_reuter.sort_values(by=['Date'])
df_reuter.head()

,OILFUTURES=,USFFTARGET= CHT,USD=,XAU=,CNPMIC=ECI,JPRPMI=ECI,KRBUSI=ECI,USPMI=ECI,WTI=,GCI=,CCE=STOCK
Date,,,,,,,,,,,
1998-06-30,14.18,5.5,1370,296.7,NaN,NaN,NaN,48.9,14.3,NaN,NaN
1998-07-31,14.21,5.5,1229,286.2,NaN,NaN,NaN,49.2,14.27,NaN,NaN
1998-08-31,13.34,5.5,1348,276.4,NaN,NaN,NaN,49.3,13.29,NaN,NaN
1998-09-30,16.14,5.25,1388.5,296.7,NaN,NaN,NaN,48.7,16.19,NaN,NaN
1998-10-31,14.42,5,1318,292.3,NaN,NaN,NaN,48.7,14.48,NaN,NaN


In [10]:

# df_reuter.shaped
# df_reuter.interpolate(method='linear', limit_direction='both', axis=0)
for c in df_reuter:
    df_reuter[c] = df_reuter[c].astype('float')
    df_reuter[c] = df_reuter[c].interpolate(limit_direction='both')
#     df_reuter[c].dtypes
df_reuter.head()
# for c in df_reuter:
#    df_reuter[c] = df_reuter[c].interpolate(limit_direction='both')

,OILFUTURES=,USFFTARGET= CHT,USD=,XAU=,CNPMIC=ECI,JPRPMI=ECI,KRBUSI=ECI,USPMI=ECI,WTI=,GCI=,CCE=STOCK
Date,,,,,,,,,,,
1998-06-30,14.18,5.50,1370.0,296.7,49.4,51.7,87.0,48.9,14.30,106.283359,5.05
1998-07-31,14.21,5.50,1229.0,286.2,49.4,51.7,87.0,49.2,14.27,106.283359,5.05
1998-08-31,13.34,5.50,1348.0,276.4,49.4,51.7,87.0,49.3,13.29,106.283359,5.05
1998-09-30,16.14,5.25,1388.5,296.7,49.4,51.7,87.0,48.7,16.19,106.283359,5.05
1998-10-31,14.42,5.00,1318.0,292.3,49.4,51.7,87.0,48.7,14.48,106.283359,5.05


In [11]:
#df_reuter = df_reuter.loc['2015-6':'2018-6']
df_reuter = df_reuter.resample('W-FRI').bfill()
df_reuter.head()

,OILFUTURES=,USFFTARGET= CHT,USD=,XAU=,CNPMIC=ECI,JPRPMI=ECI,KRBUSI=ECI,USPMI=ECI,WTI=,GCI=,CCE=STOCK
Date,,,,,,,,,,,
1998-07-03,14.21,5.5,1229.0,286.2,49.4,51.7,87.0,49.2,14.27,106.283359,5.05
1998-07-10,14.21,5.5,1229.0,286.2,49.4,51.7,87.0,49.2,14.27,106.283359,5.05
1998-07-17,14.21,5.5,1229.0,286.2,49.4,51.7,87.0,49.2,14.27,106.283359,5.05
1998-07-24,14.21,5.5,1229.0,286.2,49.4,51.7,87.0,49.2,14.27,106.283359,5.05
1998-07-31,14.21,5.5,1229.0,286.2,49.4,51.7,87.0,49.2,14.27,106.283359,5.05


In [12]:
#인코딩 후 저장
df_reuter.to_csv(var_output_path, encoding='utf-8')

# 4. Reuter 석탄 가격 지표

In [13]:
df_pre_reuter_coal = pd.read_excel(idx_path)
df_reu_coal1 = df_pre_reuter_coal.iloc[1:, 0:2]
df_reu_coal1.columns = ['Date', 'China Export Coal Current Price']
df_reu_coal2 = df_pre_reuter_coal.iloc[1:, 3:5]
df_reu_coal2.columns = ['Date', 'China Export Other Coal Volume']
df_reu_coal3 = df_pre_reuter_coal.iloc[1:, 6:8]
df_reu_coal3.columns = ['Date', 'CHina Import Coal Volume']
df_reu_coal4 = df_pre_reuter_coal.iloc[1:, 9:11]
df_reu_coal4.columns = ['Date', 'China Import Coal Coke&Briquettes Current Price']
df_reu_coal5 = df_pre_reuter_coal.iloc[1:, 12:14]
df_reu_coal5.columns = ['Date', 'India Import Coal Coke&Briquettes Current Price']
df_reu_coal6 = df_pre_reuter_coal.iloc[1:, 15:17]
df_reu_coal6.columns = ['Date', 'SouthAfrica Producer Price Mining Coal&Gas Price']
df_reu_coal = pd.merge(df_reu_coal1, df_reu_coal2, how='left')
df_reu_coal = pd.merge(df_reu_coal, df_reu_coal3, how='left')
df_reu_coal = pd.merge(df_reu_coal, df_reu_coal4, how='left')
df_reu_coal = pd.merge(df_reu_coal, df_reu_coal5, how='left')
df_reu_coal = pd.merge(df_reu_coal, df_reu_coal6, how='left')
df_reu_coal = df_reu_coal.set_index('Date')
df_reu_coal.head()

,China Export Coal Current Price,China Export Other Coal Volume,CHina Import Coal Volume,China Import Coal Coke&Briquettes Current Price,India Import Coal Coke&Briquettes Current Price,SouthAfrica Producer Price Mining Coal&Gas Price
Date,,,,,,
1998-06-30,62138000,1980000,80000,3191000,NaN,NaN
1998-07-31,66274000,2040000,140000,5564000,NaN,NaN
1998-08-31,90068000,2690000,280000,12534000,NaN,NaN
1998-09-30,82114000,2510000,170000,7609000,NaN,NaN
1998-10-31,83505000,2590000,210000,8872000,NaN,NaN


In [14]:
for c in df_reu_coal:
    df_reu_coal[c] = df_reu_coal[c].astype('float')
    df_reu_coal[c] = df_reu_coal[c].interpolate(limit_direction='both')

df_reu_coal = df_reu_coal.resample('W-FRI').bfill()

In [15]:
#인코딩 후 저장
df_reu_coal.to_csv(idx_output_path, encoding='utf-8')

# 5. 데이터 마트 통합

In [70]:
df_mart = pd.merge(df_gci['2007' : '2018-5-25'].reset_index(), pd.read_csv(pds_output_path, index_col=0, parse_dates=True, encoding='ms949').reset_index(), how='inner')
df_mart = pd.merge(df_mart, df_reuter.reset_index(), how='left')
df_mart = pd.merge(df_mart, df_reu_coal.reset_index(), how='left')
df_mart = df_mart.set_index('Date')
df_mart.head()

,NEWC,WorldBank-Agriculture Index,IMF-Crude Oil(petroleum) Price index,CRB BLS Spot Index,철근 [SHFE] 2018.10,LMEX Index,GSCI Light Energy Index (CPW 4),GSCI Livestock,WorldBank-Energy Index,중후판 12(mm)x8(ft)x20(ft) [국내(1차 유통가)] 현물,...,USPMI=ECI,WTI=,GCI=,CCE=STOCK,China Export Coal Current Price,China Export Other Coal Volume,CHina Import Coal Volume,China Import Coal Coke&Briquettes Current Price,India Import Coal Coke&Briquettes Current Price,SouthAfrica Producer Price Mining Coal&Gas Price
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-05,51.53,73.6850,102.4100,359.17,3577.0,3459.10,550.84,267.27,74.755,700000.0,...,50.4,58.17,545.876783,5.91,199838000.0,3290000.0,4700000.0,206121000.0,1.724820e+10,79.7
2007-01-12,51.24,74.1500,104.3000,363.91,3577.0,3439.80,550.84,267.27,76.300,700000.0,...,50.4,58.17,545.876783,5.91,199838000.0,3290000.0,4700000.0,206121000.0,1.724820e+10,79.7
2007-01-19,50.67,74.6150,106.1900,367.04,3577.0,3579.16,550.84,267.27,77.845,700000.0,...,50.4,58.17,545.876783,5.91,199838000.0,3290000.0,4700000.0,206121000.0,1.724820e+10,79.7
2007-01-26,50.76,75.0800,108.0800,371.99,3577.0,3465.72,550.84,267.27,79.390,700000.0,...,50.4,58.17,545.876783,5.91,199838000.0,3290000.0,4700000.0,206121000.0,1.724820e+10,79.7
2007-02-02,51.92,75.0525,109.5225,365.30,3577.0,3378.16,550.84,267.27,79.930,700000.0,...,54.1,61.78,571.881122,7.29,252353000.0,4420000.0,3930000.0,191644000.0,1.699610e+10,79.7


 -데이터 마트 메타 생성

In [71]:
df_mart_meta = pd.DataFrame(df_mart.columns, columns = ['Columns Name'])
df_mart_meta.to_csv('../data2/data_mart_meta.csv', encoding='ms949')

In [72]:
df_mart.to_csv('../data2/data_mart.csv', encoding='ms949')
df_mart.head()

,NEWC,WorldBank-Agriculture Index,IMF-Crude Oil(petroleum) Price index,CRB BLS Spot Index,철근 [SHFE] 2018.10,LMEX Index,GSCI Light Energy Index (CPW 4),GSCI Livestock,WorldBank-Energy Index,중후판 12(mm)x8(ft)x20(ft) [국내(1차 유통가)] 현물,...,USPMI=ECI,WTI=,GCI=,CCE=STOCK,China Export Coal Current Price,China Export Other Coal Volume,CHina Import Coal Volume,China Import Coal Coke&Briquettes Current Price,India Import Coal Coke&Briquettes Current Price,SouthAfrica Producer Price Mining Coal&Gas Price
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-05,51.53,73.6850,102.4100,359.17,3577.0,3459.10,550.84,267.27,74.755,700000.0,...,50.4,58.17,545.876783,5.91,199838000.0,3290000.0,4700000.0,206121000.0,1.724820e+10,79.7
2007-01-12,51.24,74.1500,104.3000,363.91,3577.0,3439.80,550.84,267.27,76.300,700000.0,...,50.4,58.17,545.876783,5.91,199838000.0,3290000.0,4700000.0,206121000.0,1.724820e+10,79.7
2007-01-19,50.67,74.6150,106.1900,367.04,3577.0,3579.16,550.84,267.27,77.845,700000.0,...,50.4,58.17,545.876783,5.91,199838000.0,3290000.0,4700000.0,206121000.0,1.724820e+10,79.7
2007-01-26,50.76,75.0800,108.0800,371.99,3577.0,3465.72,550.84,267.27,79.390,700000.0,...,50.4,58.17,545.876783,5.91,199838000.0,3290000.0,4700000.0,206121000.0,1.724820e+10,79.7
2007-02-02,51.92,75.0525,109.5225,365.30,3577.0,3378.16,550.84,267.27,79.930,700000.0,...,54.1,61.78,571.881122,7.29,252353000.0,4420000.0,3930000.0,191644000.0,1.699610e+10,79.7


# 6. 정규화

In [19]:
def normalize_(df):
    values = df.values
    values = np.asarray(values, dtype=np.float32)

    scaler = MinMaxScaler(feature_range=(0, 1))
    values = scaler.fit_transform(values)
    df = pd.DataFrame(values, index=df.index, columns=df.columns)

    return df, scaler

In [20]:
# df_reu_coal_scaler.transform(df_reu_coal)
# df_reu_coal_scaler.data_min_
# df_reu_coal_scaler.data_max_
df_gci_normalize, df_gci_scaler = normalize_(df_gci)
df_pds_normalize, df_pds_scaler = normalize_(df_pds)
df_reuter_normalize, df_reuter_scaler = normalize_(df_reuter)
df_reu_coal_normalize, df_reu_coal_scaler = normalize_(df_reu_coal)

In [21]:
# np.savetxt('../data2/scaler_test.csv', df_reu_coal_scaler.inverse_transform(df_reu_coal_normalize),  delimiter=',')
np.savetxt('../data2/weekly_gci_normalize.csv', df_gci_normalize,  delimiter=',')
np.savetxt('../data2/weekly_pds_normalize.csv', df_pds_normalize,  delimiter=',')
np.savetxt('../data2/var_reuter_set_normalize.csv', df_reuter_normalize,  delimiter=',')
np.savetxt('../data2/idx_reuter_coal_normalize.csv', df_reu_coal_normalize,  delimiter=',')

In [81]:
df_mart_normalize, df_mart_scaler = normalize_(df_mart)
np.savetxt('../data2/data_mart_normalize.csv', df_mart_normalize, delimiter=',')

# 7. 상관계수 계산

데이터마트 상관계수 저장

In [82]:
df_mart_ni = df_mart
for c in df_mart_ni:
    df_mart_ni[c] = df_mart_ni[c].astype('float')
df_mart_ni = df_mart_ni.reset_index()
# df_mart_ni = df_mart_ni.set_index('Date')
df_mart_ni

,Date,NEWC,WorldBank-Agriculture Index,IMF-Crude Oil(petroleum) Price index,CRB BLS Spot Index,철근 [SHFE] 2018.10,LMEX Index,GSCI Light Energy Index (CPW 4),GSCI Livestock,WorldBank-Energy Index,...,USPMI=ECI,WTI=,GCI=,CCE=STOCK,China Export Coal Current Price,China Export Other Coal Volume,CHina Import Coal Volume,China Import Coal Coke&Briquettes Current Price,India Import Coal Coke&Briquettes Current Price,SouthAfrica Producer Price Mining Coal&Gas Price
0,2007-01-05,0.028219,0.000000,0.239450,0.206989,0.548731,0.650825,0.785111,0.297407,0.257692,...,0.607714,0.260756,0.420452,0.142310,0.235890,0.466089,0.074988,0.014594,0.016201,0.005970
1,2007-01-12,0.026252,0.008248,0.249212,0.224154,0.548731,0.643924,0.785111,0.297407,0.269315,...,0.607714,0.260756,0.420452,0.142310,0.235890,0.466089,0.074988,0.014594,0.016201,0.005970
2,2007-01-19,0.022385,0.016497,0.258974,0.235488,0.548731,0.693750,0.785111,0.297407,0.280937,...,0.607714,0.260756,0.420452,0.142310,0.235890,0.466089,0.074988,0.014594,0.016201,0.005970
3,2007-01-26,0.022996,0.024745,0.268736,0.253413,0.548731,0.653191,0.785111,0.297407,0.292560,...,0.607714,0.260756,0.420452,0.142310,0.235890,0.466089,0.074988,0.014594,0.016201,0.005970
4,2007-02-02,0.030864,0.024257,0.276187,0.229187,0.548731,0.621886,0.785111,0.297407,0.296622,...,0.749131,0.293384,0.446203,0.201669,0.301645,0.629149,0.052165,0.009668,0.013006,0.005970
5,2007-02-09,0.040768,0.023769,0.283637,0.249937,0.548731,0.673828,0.785111,0.297407,0.300685,...,0.749131,0.293384,0.446203,0.201669,0.301645,0.629149,0.052165,0.009668,0.013006,0.005970
6,2007-02-16,0.057116,0.023282,0.291088,0.261850,0.548731,0.722581,0.785111,0.297407,0.304747,...,0.749131,0.293384,0.446203,0.201669,0.301645,0.629149,0.052165,0.009668,0.013006,0.005970
7,2007-02-23,0.041378,0.022794,0.298538,0.307442,0.548731,0.750454,0.785111,0.297407,0.308809,...,0.749131,0.293384,0.446203,0.201669,0.301645,0.629149,0.052165,0.009668,0.013006,0.005970
8,2007-03-02,0.031678,0.025765,0.309424,0.282383,0.548731,0.721802,0.785111,0.297407,0.318664,...,0.699444,0.330983,0.478926,0.247263,0.250420,0.526696,0.103738,0.043033,0.083754,0.005970
9,2007-03-09,0.034527,0.028736,0.320309,0.288466,0.548731,0.779035,0.785111,0.297407,0.328519,...,0.699444,0.330983,0.478926,0.247263,0.250420,0.526696,0.103738,0.043033,0.083754,0.005970


In [83]:
corr = df_mart_ni.corr(method='pearson')
corr.head()

,NEWC,WorldBank-Agriculture Index,IMF-Crude Oil(petroleum) Price index,CRB BLS Spot Index,철근 [SHFE] 2018.10,LMEX Index,GSCI Light Energy Index (CPW 4),GSCI Livestock,WorldBank-Energy Index,중후판 12(mm)x8(ft)x20(ft) [국내(1차 유통가)] 현물,...,USPMI=ECI,WTI=,GCI=,CCE=STOCK,China Export Coal Current Price,China Export Other Coal Volume,CHina Import Coal Volume,China Import Coal Coke&Briquettes Current Price,India Import Coal Coke&Briquettes Current Price,SouthAfrica Producer Price Mining Coal&Gas Price
NEWC,1.000000,0.626433,0.565747,0.528125,0.623158,0.551451,0.435339,-0.251066,0.653176,0.540678,...,0.122820,0.621396,0.557278,0.390943,0.507305,0.203382,-0.137353,0.175240,-0.227293,-0.125119
WorldBank-Agriculture Index,0.626433,1.000000,0.821156,0.870087,0.528731,0.542732,0.476732,0.241859,0.796583,0.659875,...,0.197843,0.748958,0.448096,0.256154,0.127043,-0.157039,0.221687,0.489389,-0.006527,-0.292044
IMF-Crude Oil(petroleum) Price index,0.565747,0.821156,1.000000,0.726644,0.546641,0.649235,0.713119,0.290113,0.978099,0.745361,...,0.128938,0.960669,0.645324,0.454326,0.298469,0.155256,0.044567,0.311920,-0.247789,-0.449272
CRB BLS Spot Index,0.528125,0.870087,0.726644,1.000000,0.463413,0.692741,0.291321,0.349493,0.715291,0.437518,...,0.544399,0.678487,0.385402,0.159667,-0.061935,-0.236211,0.348790,0.556881,0.167011,-0.036803
철근 [SHFE] 2018.10,0.623158,0.528731,0.546641,0.463413,1.000000,0.634316,0.755712,-0.273477,0.590147,0.731325,...,0.232971,0.553080,0.633700,0.482845,0.372028,0.203477,-0.126762,0.283764,-0.515267,-0.304658


np.savetxt('../data2/data_mart_correlation.csv', corr, delimiter=',')

데이터마트 정규화 값 상관계수 저장

In [82]:
df_mart_ni = df_mart_normalize
for c in df_mart_ni:
    df_mart_ni[c] = df_mart_ni[c].astype('float')
df_mart_ni = df_mart_ni.reset_index()
# df_mart_ni = df_mart_ni.set_index('Date')
df_mart_ni

,Date,NEWC,WorldBank-Agriculture Index,IMF-Crude Oil(petroleum) Price index,CRB BLS Spot Index,철근 [SHFE] 2018.10,LMEX Index,GSCI Light Energy Index (CPW 4),GSCI Livestock,WorldBank-Energy Index,...,USPMI=ECI,WTI=,GCI=,CCE=STOCK,China Export Coal Current Price,China Export Other Coal Volume,CHina Import Coal Volume,China Import Coal Coke&Briquettes Current Price,India Import Coal Coke&Briquettes Current Price,SouthAfrica Producer Price Mining Coal&Gas Price
0,2007-01-05,0.028219,0.000000,0.239450,0.206989,0.548731,0.650825,0.785111,0.297407,0.257692,...,0.607714,0.260756,0.420452,0.142310,0.235890,0.466089,0.074988,0.014594,0.016201,0.005970
1,2007-01-12,0.026252,0.008248,0.249212,0.224154,0.548731,0.643924,0.785111,0.297407,0.269315,...,0.607714,0.260756,0.420452,0.142310,0.235890,0.466089,0.074988,0.014594,0.016201,0.005970
2,2007-01-19,0.022385,0.016497,0.258974,0.235488,0.548731,0.693750,0.785111,0.297407,0.280937,...,0.607714,0.260756,0.420452,0.142310,0.235890,0.466089,0.074988,0.014594,0.016201,0.005970
3,2007-01-26,0.022996,0.024745,0.268736,0.253413,0.548731,0.653191,0.785111,0.297407,0.292560,...,0.607714,0.260756,0.420452,0.142310,0.235890,0.466089,0.074988,0.014594,0.016201,0.005970
4,2007-02-02,0.030864,0.024257,0.276187,0.229187,0.548731,0.621886,0.785111,0.297407,0.296622,...,0.749131,0.293384,0.446203,0.201669,0.301645,0.629149,0.052165,0.009668,0.013006,0.005970
5,2007-02-09,0.040768,0.023769,0.283637,0.249937,0.548731,0.673828,0.785111,0.297407,0.300685,...,0.749131,0.293384,0.446203,0.201669,0.301645,0.629149,0.052165,0.009668,0.013006,0.005970
6,2007-02-16,0.057116,0.023282,0.291088,0.261850,0.548731,0.722581,0.785111,0.297407,0.304747,...,0.749131,0.293384,0.446203,0.201669,0.301645,0.629149,0.052165,0.009668,0.013006,0.005970
7,2007-02-23,0.041378,0.022794,0.298538,0.307442,0.548731,0.750454,0.785111,0.297407,0.308809,...,0.749131,0.293384,0.446203,0.201669,0.301645,0.629149,0.052165,0.009668,0.013006,0.005970
8,2007-03-02,0.031678,0.025765,0.309424,0.282383,0.548731,0.721802,0.785111,0.297407,0.318664,...,0.699444,0.330983,0.478926,0.247263,0.250420,0.526696,0.103738,0.043033,0.083754,0.005970
9,2007-03-09,0.034527,0.028736,0.320309,0.288466,0.548731,0.779035,0.785111,0.297407,0.328519,...,0.699444,0.330983,0.478926,0.247263,0.250420,0.526696,0.103738,0.043033,0.083754,0.005970


In [83]:
corr = df_mart_ni.corr(method='pearson')
corr.head()

,NEWC,WorldBank-Agriculture Index,IMF-Crude Oil(petroleum) Price index,CRB BLS Spot Index,철근 [SHFE] 2018.10,LMEX Index,GSCI Light Energy Index (CPW 4),GSCI Livestock,WorldBank-Energy Index,중후판 12(mm)x8(ft)x20(ft) [국내(1차 유통가)] 현물,...,USPMI=ECI,WTI=,GCI=,CCE=STOCK,China Export Coal Current Price,China Export Other Coal Volume,CHina Import Coal Volume,China Import Coal Coke&Briquettes Current Price,India Import Coal Coke&Briquettes Current Price,SouthAfrica Producer Price Mining Coal&Gas Price
NEWC,1.000000,0.626433,0.565747,0.528125,0.623158,0.551451,0.435339,-0.251066,0.653176,0.540678,...,0.122820,0.621396,0.557278,0.390943,0.507305,0.203382,-0.137353,0.175240,-0.227293,-0.125119
WorldBank-Agriculture Index,0.626433,1.000000,0.821156,0.870087,0.528731,0.542732,0.476732,0.241859,0.796583,0.659875,...,0.197843,0.748958,0.448096,0.256154,0.127043,-0.157039,0.221687,0.489389,-0.006527,-0.292044
IMF-Crude Oil(petroleum) Price index,0.565747,0.821156,1.000000,0.726644,0.546641,0.649235,0.713119,0.290113,0.978099,0.745361,...,0.128938,0.960669,0.645324,0.454326,0.298469,0.155256,0.044567,0.311920,-0.247789,-0.449272
CRB BLS Spot Index,0.528125,0.870087,0.726644,1.000000,0.463413,0.692741,0.291321,0.349493,0.715291,0.437518,...,0.544399,0.678487,0.385402,0.159667,-0.061935,-0.236211,0.348790,0.556881,0.167011,-0.036803
철근 [SHFE] 2018.10,0.623158,0.528731,0.546641,0.463413,1.000000,0.634316,0.755712,-0.273477,0.590147,0.731325,...,0.232971,0.553080,0.633700,0.482845,0.372028,0.203477,-0.126762,0.283764,-0.515267,-0.304658


In [85]:
np.savetxt('../data2/data_mart_normalize_correlation.csv', corr, delimiter=',')